## User guide
https://pcse.readthedocs.io/en/stable/user_guide.html#getting-started


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pp
from datetime import datetime, timedelta


In [ ]:
crop_name = "rice"
variety_name = "Rice_IR8A"
campaign_start_date = "2021-01-01"
crop_start_date = "2021-01-01"
crop_end_date = "2021-04-01"
irrigation_events = {
    "2021-02-14": 10,
#     "2006-08-01": 10,
}

city = "Cotonou"
latitude = 51.97
longitude = 5.67


## Running PCSE/WOFOST with custom input data
For running PCSE/WOFOST (and PCSE models in general) with your own data sources you need three different types of inputs:

1. Model parameters that parameterize the different model components. These parameters usually consist of :
   - a set of crop parameters (or multiple sets in case of crop rotations), 
   - a set of soil parameters and 
   - a set of site parameters: ancillary parameters that are specific for a location.

2. Driving variables represented by weather data which can be derived from various sources.
3. Agromanagement actions which specify the farm activities that will take place on the field that is simulated by PCSE.

In [ ]:
from pcse.fileinput import CABOFileReader, YAMLCropDataProvider
from pathlib import Path

from utils import get_soil_data

## 1. Model parameters
### 1.1. Crop parameters

In [ ]:
cropfile = "https://raw.githubusercontent.com/ajwdewit/WOFOST_crop_parameters/master/"
cropdata = YAMLCropDataProvider(repository=cropfile)

In [ ]:
pp(cropdata.get_crops_varieties())

### 1.2. Soil parameters
- soil type 
- soil physical properties

In [ ]:
# we will use the water balance for 
# freely draining soils and use the soil file for medium fine sand
soildata = get_soil_data()

In [ ]:
pp(soildata)

### 1.3. Site parameters

In [ ]:
from pcse.util import WOFOST71SiteDataProvider

In [ ]:
# the initial conditions of the water balance such as the initial soil moisture content
sitedata = WOFOST71SiteDataProvider(
    WAV=100, # initial soil moisture content
    CO2=360, # the atmospheric CO2 concentration
)

In [ ]:
expected_sitedata = {
    'IFUNRN': 0,
    'NOTINF': 0,
    'SSI': 0.0, # the initial surface storage
    'SSMAX': 0.0, # the maximum surface storage
    'WAV': 100.0,
    'SMLIM': 0.4
}
assert sitedata == expected_sitedata

### 1.4. Combine them

In [ ]:
from pcse.base import ParameterProvider

In [ ]:
parameters = ParameterProvider(cropdata=cropdata, soildata=soildata, sitedata=sitedata)

## 2. Weather data

In [ ]:
from pcse.db import NASAPowerWeatherDataProvider

In [ ]:
wdp = NASAPowerWeatherDataProvider(
    latitude=latitude, longitude=longitude,
)

In [ ]:
print(wdp)

In [ ]:
crop_start_datetime = datetime.strptime(crop_start_date, "%Y-%m-%d").date()
crop_end_datetime = datetime.strptime(crop_end_date, "%Y-%m-%d").date()

In [ ]:
df_weatherdataprovider = pd.DataFrame(wdp.export()).set_index("DAY")

fig, ax = plt.subplots(1, 1, figsize=(15,5))
mask = (df_weatherdataprovider.index >= crop_start_datetime) & (df_weatherdataprovider.index <= crop_end_datetime)
df_weatherdataprovider.loc[mask, ["RAIN"]].plot(ax=ax)

## 3. Agromanagement

https://github.com/ajwdewit/pcse_notebooks/blob/master/06_advanced_agromanagement_with_PCSE.ipynb

In [ ]:
import yaml
from irrigation import (
    update_agromanagement,
    get_timed_events,
    add_irrigation_event,
    get_irrigation_events,
    get_crop_start_date
)

In [ ]:
yaml_agro_template = f"""
- {campaign_start_date}:
    CropCalendar:
        crop_name: {crop_name}
        variety_name: {variety_name}
        crop_start_date: {crop_start_date}
        crop_start_type: emergence
        crop_end_date: {crop_end_date}
        crop_end_type: harvest
        max_duration: 300
    TimedEvents: null
    StateEvents: null
"""

agromanagement = yaml.safe_load(yaml_agro_template)

In [ ]:
timed_events = get_timed_events(agromanagement)

for date, volume in irrigation_events.items():
    timed_events = add_irrigation_event(date, volume, timed_events)

agromanagement_irrigation = update_agromanagement(agromanagement, timed_events)

In [ ]:
first_campaign = agromanagement_irrigation[0]
irrigation_events = get_irrigation_events(first_campaign)
irrigation_events

## 4. Initializing WOFOST model

In [ ]:
from pcse.models import Wofost72_WLP_FD, Wofost72_PP

In [ ]:
wofost_sim = Wofost72_WLP_FD(parameters, wdp, agromanagement)
wofost_irrigation_sim = Wofost72_WLP_FD(parameters, wdp, agromanagement_irrigation)

In [ ]:
wofost_sim.run_till_terminate()
wofost_irrigation_sim.run_till_terminate()

In [ ]:
outputs = wofost_sim.get_output()
df_output = pd.DataFrame(outputs)

outputs_irrigation = wofost_irrigation_sim.get_output()
df_output_irrigation = pd.DataFrame(outputs_irrigation)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15, 5))
var = "LAI"
df_output.set_index("day")[var].plot(ax=ax)
df_output_irrigation.set_index("day")[var].plot(ax=ax, label=f"{var} irrigation")
for event_cfg in irrigation_events:
    event_date, irrigation_water_in_cm = event_cfg
    ax.axvline(x=event_date, c="red", label=f"irrigation event: {irrigation_water_in_cm['amount']} cm")

# df_weatherdataprovider.loc[mask, ["RAIN"]].plot(ax=ax)
plt.title(f"{crop_name} {variety_name} at {city} ({latitude=}, {longitude=})")
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15, 5))
var = "SM"
df_output.set_index("day")[var].plot(ax=ax)
df_output_irrigation.set_index("day")[var].plot(ax=ax, label=f"{var} irrigation")

for event_cfg in irrigation_events:
    event_date, irrigation_water_in_cm = event_cfg
    ax.axvline(x=event_date, c="red", label=f"irrigation event: {irrigation_water_in_cm['amount']} cm")

# df_weatherdataprovider.loc[mask, ["RAIN"]].plot(ax=ax)
plt.legend()
plt.show()

## Optimize a complex irrigation planning
### 1. `scipy` implementation of BFGS

In [ ]:
import scipy
import numpy as np

In [ ]:
def objective_function(
    search_params: np.ndarray,
    parameterprovider=parameters,
    weatherdataprovider=wdp,
    agromanagement=agromanagement,
) -> float:
    assert len(search_params) == 2
    delay_in_days, volume = search_params
    crop_start_date = get_crop_start_date(agromanagement)
    irrigation_date = crop_start_date + timedelta(days=delay_in_days)

    timed_events = get_timed_events(agromanagement)
    timed_events = add_irrigation_event(irrigation_date.strftime("%Y-%m-%d"), volume, timed_events)
    agromanagement_irrigation = update_agromanagement(agromanagement, timed_events)
    
    wofost = Wofost72_WLP_FD(parameterprovider, weatherdataprovider, agromanagement_irrigation)
    wofost.run_till_terminate()
    output = wofost.get_output()
    df_output = pd.DataFrame(output)
    return -df_output["LAI"].sum()

In [ ]:
irrigation_volumes = np.arange(start=10, stop=20, step=1)
loss = [objective_function((90, irrigation_volume)) for irrigation_volume in irrigation_volumes]

In [ ]:
plt.plot(irrigation_volumes, loss)

In [ ]:
scipy.optimize.minimize(
    fun=objective_function,
    x0=(90, 5)
)

### 2. Implementation of a derivative-free local optimization algorithm called Supblex.

In [ ]:
import nlopt

In [ ]:
class ModelRerunner(object):
    parameters = ["delay_in_days", "volume"]
    
    def __init__(self, params, wdp, agro):
        self.params = params
        self.wdp = wdp
        self.agro = agro
        
    def __call__(self, par_values):
        # Check if correct number of parameter values were provided
        if len(par_values) != len(self.parameters):
            msg = "Optimizing %i parameters, but only % values were provided!" % (len(self.parameters, len(par_values)))
            raise RuntimeError(msg)

        # Clear any existing overrides
        self.params.clear_override()

            
        # Irrigation
        delay_in_days, volume = par_values
        crop_start_date = get_crop_start_date(self.agro)
        irrigation_date = crop_start_date + timedelta(days=delay_in_days)

        timed_events = get_timed_events(self.agro)
        timed_events = add_irrigation_event(irrigation_date.strftime("%Y-%m-%d"), volume, timed_events)
        agromanagement_irrigation = update_agromanagement(agromanagement, timed_events)

        # Run the model with given parameter values
        wofost = Wofost72_WLP_FD(self.params, self.wdp, agromanagement_irrigation)
        wofost.run_till_terminate()
        df = pd.DataFrame(wofost.get_output())
        df.index = pd.to_datetime(df.day)
        return df

In [ ]:
class ObjectiveFunctionCalculator(object):
    """Computes the objective function.
    
    This class runs the simulation model with given parameter values and returns the objective
    function as the sum of squared difference between observed and simulated LAI.
.   """
    
    def __init__(self, params, wdp, agro):
        self.modelrerunner = ModelRerunner(params, wdp, agro)
        self.n_calls = 0
       
    def __call__(self, par_values, grad=None):
        """Runs the model and computes the objective function for given par_values.
        
        The input parameter 'grad' must be defined in the function call, but is only
        required for optimization methods where analytical gradients can be computed.
        """
        self.n_calls += 1
        print(".", end="")

        # Run the model and collect output
        self.df_simulations = self.modelrerunner(par_values)
        obj_func = -self.df_simulations["LAI"].sum()
        return obj_func

In [ ]:
opt = nlopt.opt(nlopt.LN_SBPLX, 2)

# Assign the objective function calculator
objfunc_calculator = ObjectiveFunctionCalculator(parameters, wdp, agromanagement)
opt.set_min_objective(objfunc_calculator)

# lower bounds of parameters values
opt.set_lower_bounds([0, 0])

# upper bounds of parameters values
opt.set_upper_bounds([180, 30])

# the initial step size to compute numerical gradients
opt.set_initial_step([1, 1.])

# Maximum number of evaluations allowed
opt.set_maxeval(200)

# Relative tolerance for convergence
opt.set_ftol_rel(0.1)

# Start the optimization with the first guess
firstguess = (60, 5)

x = opt.optimize(firstguess)

In [ ]:
opt_delay_in_days, opt_volume = x
print(f"\noptimum at {opt_delay_in_days=}, {opt_volume=}")
print(f"minimum value = {opt.last_optimum_value()}")
print(f"result code = {opt.last_optimize_result()}")
print(f"With {objfunc_calculator.n_calls} function calls")

In [ ]:
crop_start_date = get_crop_start_date(agromanagement)
irrigation_date = crop_start_date + timedelta(days=opt_delay_in_days)

timed_events = get_timed_events(agromanagement)
timed_events = add_irrigation_event(irrigation_date.strftime("%Y-%m-%d"), opt_volume, timed_events)
agromanagement_irrigation = update_agromanagement(agromanagement, timed_events)

# Run the model with given parameter values
wofost = Wofost72_WLP_FD(parameters, wdp, agromanagement_irrigation)
wofost.run_till_terminate()
df_output_irrigation_opt = pd.DataFrame(wofost.get_output())

In [ ]:
first_campaign = agromanagement_irrigation[0]
irrigation_events = get_irrigation_events(first_campaign)
irrigation_events

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15, 5))
var = "LAI"
df_output.set_index("day")[var].plot(ax=ax)
df_output_irrigation_opt.set_index("day")[var].plot(ax=ax, label=f"{var} irrigation")
for event_cfg in irrigation_events:
    event_date, irrigation_water_in_cm = event_cfg
    ax.axvline(x=event_date, c="red", label=f"irrigation event: {irrigation_water_in_cm['amount']:.2f} cm")

plt.title(f"{crop_name} {variety_name} at {city} ({latitude=}, {longitude=}): optimal date and volume")
plt.legend()
plt.show()

In [ ]:
def objective_function_on_sphere(
    search_params: np.ndarray,
    parameterprovider=parameters,
    weatherdataprovider=wdp,
    agromanagement=agromanagement,
) -> float:
    assert len(search_params) == 2
    delay_in_days, volume = search_params
    crop_start_date = get_crop_start_date(agromanagement)
    irrigation_date = crop_start_date + timedelta(days=int(abs(delay_in_days * 30 * 6)))

    timed_events = get_timed_events(agromanagement)
    timed_events = add_irrigation_event(irrigation_date.strftime("%Y-%m-%d"), abs(volume * 30), timed_events)
    agromanagement_irrigation = update_agromanagement(agromanagement, timed_events)
    
    wofost = Wofost72_WLP_FD(parameterprovider, weatherdataprovider, agromanagement_irrigation)
    wofost.run_till_terminate()
    output = wofost.get_output()
    df_output = pd.DataFrame(output)
    return -df_output["LAI"].sum()

In [ ]:
import nevergrad as ng

optimizer = ng.optimizers.NGOpt(parametrization=2, budget=1000)
recommendation = optimizer.minimize(objective_function_on_sphere)

print(recommendation.value)  # recommended value

In [ ]:
delay_in_days_normalized, volume_normalized = recommendation.value
delay_in_days = int(abs(delay_in_days_normalized * 30 * 6))
volume = abs(volume_normalized * 30)

In [ ]:
crop_start_date = get_crop_start_date(agromanagement)
irrigation_date = crop_start_date + timedelta(days=opt_delay_in_days)

timed_events = get_timed_events(agromanagement)
timed_events = add_irrigation_event(irrigation_date.strftime("%Y-%m-%d"), opt_volume, timed_events)
agromanagement_irrigation = update_agromanagement(agromanagement, timed_events)

# Run the model with given parameter values
wofost = Wofost72_WLP_FD(parameters, wdp, agromanagement_irrigation)
wofost.run_till_terminate()
df_output_irrigation_opt = pd.DataFrame(wofost.get_output())

In [ ]:
first_campaign = agromanagement_irrigation[0]
irrigation_events = get_irrigation_events(first_campaign)
irrigation_events

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15, 5))
var = "LAI"
df_output.set_index("day")[var].plot(ax=ax)
df_output_irrigation_opt.set_index("day")[var].plot(ax=ax, label=f"{var} irrigation")
for event_cfg in irrigation_events:
    event_date, irrigation_water_in_cm = event_cfg
    ax.axvline(x=event_date, c="red", label=f"irrigation event: {irrigation_water_in_cm['amount']:.2f} cm")

plt.title(f"{crop_name} {variety_name} at {city} ({latitude=}, {longitude=}): optimal date and volume")
plt.legend()
plt.show()